# Types of RNN
1. Many to one
2. One to many
3. Many to Many
4. One to One

In [1]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, TimeDistributed, RepeatVector
from keras.utils import plot_model

# ----------------------------
# 1. ONE-TO-ONE (Standard NN)
# ----------------------------

# One input → One output
model_one_to_one = Sequential()
model_one_to_one.add(Dense(64, input_shape=(100,), activation='relu'))  # e.g., 100 features
model_one_to_one.add(Dense(10, activation='softmax'))  # e.g., 10 classes
print("One-to-One Model:")
model_one_to_one.summary()


# ----------------------------
# 2. ONE-TO-MANY (e.g., Image Captioning)
# ----------------------------

# One input → Sequence output
model_one_to_many = Sequential()
model_one_to_many.add(Dense(128, input_shape=(2048,), activation='relu'))  # e.g., image feature vector
model_one_to_many.add(RepeatVector(10))  # Repeat input vector for 10 time steps
model_one_to_many.add(LSTM(64, return_sequences=True))  # LSTM to generate sequence
model_one_to_many.add(TimeDistributed(Dense(5000, activation='softmax')))  # vocab_size = 5000
print("\nOne-to-Many Model:")
model_one_to_many.summary()


# ----------------------------
# 3. MANY-TO-ONE (e.g., Sentiment Analysis)
# ----------------------------

# Sequence input → One output
model_many_to_one = Sequential()
model_many_to_one.add(LSTM(64, input_shape=(100, 50)))  # 100 time steps, 50 features
model_many_to_one.add(Dense(1, activation='sigmoid'))  # Binary output
print("\nMany-to-One Model:")
model_many_to_one.summary()


# ----------------------------
# 4A. MANY-TO-MANY (Synchronized)
# ----------------------------

# Sequence input → Sequence output (same length)
model_many_to_many_sync = Sequential()
model_many_to_many_sync.add(LSTM(64, return_sequences=True, input_shape=(20, 100)))  # 20 time steps, 100 features
model_many_to_many_sync.add(TimeDistributed(Dense(10, activation='softmax')))  # e.g., 10 tags
print("\nMany-to-Many (Synchronized) Model:")
model_many_to_many_sync.summary()


# ----------------------------
# 4B. MANY-TO-MANY (Un-Synchronized / Encoder-Decoder)
# ----------------------------

# Input sequence → Output sequence (different lengths)
# Used in machine translation

# Dimensions
encoder_timesteps = 15
decoder_timesteps = 20
input_dim = 100     # Input embedding dim
output_dim = 200    # Output vocab size

# Encoder
encoder_inputs = Input(shape=(encoder_timesteps, input_dim))
encoder_lstm = LSTM(64, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(decoder_timesteps, output_dim))
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_dim, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Final model
model_many_to_many_unsync = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print("\nMany-to-Many (Un-Synchronized) Model:")
model_many_to_many_unsync.summary()


One-to-One Model:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,114 (27.79 KB)

...

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 15, 100)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 20, 200)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 64),      │     42,240 │ input_layer_4[0]… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 20, 64),  │     67,840 │ input_layer_5[0]… │
│                     │ (None, 64),       │            │ lstm_3[0][1],     │
│                     │ (None, 64)]       │            │ lstm_3[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 20, 200)   │     13,000 │ lstm_4[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 123,080 (480.78 KB)

 Trainable params: 123,080 (480.78 KB)

 Non-trainable params: 0 (0.00 B)